# Задача 1
Задать базовый класс `BankProduct` (банковский продукт - кредит, вклад). Основными полями его будут: срок, имя, фамилия клиента, сумма (в руб.), процент.  Базовый класс должен так же уметь выводить данную информацию по продукту с помощью переопределения метода `__str__`.


Задать два класса-наследника `Credit` (кредит), `Deposit` (вклад).  
Данные классы будут расширять базовый полями:  
Для класса `Credit`:
* тип (поле класса)
* минимальная сумма досрочного погашения
* максимальный срок просрочки оплаты по кредиту (в днях)
* пени за просрочку (% в день)

Для класса `Deposit`:
* тип (поле класса)
* возможность пополнять счет
* возможность частичного снятия со счета
* минимальная дата, с которой можно закрыть счет

Далее.

Создать несколько объектов класса `Credit` и `Deposit` (по 3 на каждый класс)

В цикле подсчитать сумму выданных кредитов (в руб.) и сумму депозитов на счетах в банке (в руб.). Необходимо обращаться к соответствующим полям класса.

In [1]:
class BankProduct:
    def __init__(self, term, name, surname, summa, percent):
        self.term = term
        self.name = name
        self.surname = surname
        self.summa = summa
        self.percent = percent
        
    def __str__(self):   
        return 'Client: {} {}, Term: {}, Sum: {}, Percent: {}'.format(
            self.name, self.surname, self.term, self.summa, self.percent)

class Credit(BankProduct):
    type = 'Credit'
    def __init__(self, term, name, surname, summa, percent, 
                 min_early_repayment_sum, max_delay_term_days, late_fees):
        super().__init__(term, name, surname, summa, percent)
        self.min_early_repayment_sum = min_early_repayment_sum
        self.max_delay_term_days = max_delay_term_days
        self.late_fees = late_fees

class Deposit(BankProduct):
    type = 'Deposit'
    def __init__(self, term, name, surname, summa, percent, 
                 can_add_account, can_partial_withdraw, min_date_close_account):
        super().__init__(term, name, surname, summa, percent)
        self.can_add_account = can_add_account
        self.can_partial_withdraw = can_partial_withdraw
        self.min_date_close_account = min_date_close_account

       

In [2]:
# Создать несколько объектов класса Credit и Deposit (по 3 на каждый класс)
cr1 = Credit(12, 'ivan', 'ivanov', 500_000, 12, 250000, 10, 0.01)
cr2 = Credit(24, 'petr', 'gusev', 1_500_000, 15.5, 100000, 10, 0.01)
cr3 = Credit(36, 'oleg', 'pavlov', 200_000, 12.99, 250000, 10, 0.01)

dep1 = Deposit(12, 'anna', 'ivanova', 300_000, 6, True, True, '2019-08-01')
dep2 = Deposit(15, 'olga', 'volkova', 4_000_000, 5.6, 100000, 10, '2019-12-01')
dep3 = Deposit(18, 'anna', 'zaytseva', 700_000, 4.5, 250000, 10, '2020-03-31')


In [3]:
#В цикле подсчитать сумму выданных кредитов (в руб.) 
#и сумму депозитов на счетах в банке (в руб.). Необходимо обращаться к соответствующим полям класса.

def product_sum(*products):
    summa = 0
    for product in products:
        summa += product.summa
    return summa

In [4]:
print(product_sum(cr1, cr2, cr3))

2200000


In [5]:
print(product_sum(dep1, dep2, dep3))

5000000


# Задача 2

Для классов `Credit` и `Deposit` реализовать методы 

1. подсчета финальной суммы, которую выплатит заемщик (в случае кредита) и которую получит вкладчик (в случае депозита), с учетом капитализации, если не будет досрочных погашений (в случае кредита) и пополнений/списаний (в случае депозита).

2. подсчета возможности выдачи кредита данному клиенту. Необходимо передать в метод доп. параметр - зп клиента. Если ежемесячный платеж получается больше, чем 60% его зп, то отказать в выдаче кредита (вернуть `False`), иначе - `True`

3. `*` подсчета финальной суммы с учетом ежемесячной капитализации, которую получит вкладчик, если по истечение половины срока депозита, снимет n-ю долю со счета (параметр n должен передаться в метод как аргумент функции)

In [43]:
class Credit(BankProduct):
    type = 'Credit'
    def __init__(self, term, name, surname, summa, percent, 
                 min_early_repayment_sum, max_delay_term_days, late_fees):
        super().__init__(term, name, surname, summa, percent)
        self.min_early_repayment_sum = min_early_repayment_sum
        self.max_delay_term_days = max_delay_term_days
        self.late_fees = late_fees
    
    # Процентная ставка по кредиту в месяц
    def __get_monthly_percent(self, year_percent):       
        return (year_percent/12/100)

    # Коэффициент аннуитета
    def __get_annuity_coeff(self, term_months, year_percent):
        mp = self.__get_monthly_percent(year_percent)       
        return (mp * (1 + mp)**term_months) / ((1 + mp)**term_months - 1)
       
    # Ежемесячный аннуитетный платеж 
    def __get_monthly_payment(self, summa, annuity_coeff):
        return annuity_coeff * summa       
        
    # 1. подсчет финальной суммы, которую выплатит заемщик (в случае кредита)
    def calc_total_summa(self):
        summa = self.summa
        term_months = self.term
        year_percent = self.percent       
        return 'Общая сумма кредита: ' + str(
            self.__get_monthly_payment(summa, self.__get_annuity_coeff(term_months, year_percent)) * term_months
        )

    #2. подсчет возможности выдачи кредита данному клиенту
    def can_issue(self, client_salary):
        ac = self.__get_annuity_coeff(self.term, self.percent)
        if self.__get_monthly_payment(self.summa, ac) > client_salary*0.6:
            return False
        else:
            return True


In [44]:
cr1 = Credit(36, 'ivan', 'ivanov', 1_000_000, 20, 250000, 10, 0.01)
print(cr1.calc_total_summa())  

Общая сумма кредита: 1337889.0009506599


In [45]:
cr1.can_issue(100_000)

True

In [102]:
class Deposit(BankProduct):
    type = 'Deposit'
    def __init__(self, term, name, surname, summa, percent, 
                 can_add_account, can_partial_withdraw, min_date_close_account):
        super().__init__(term, name, surname, summa, percent)
        self.can_add_account = can_add_account
        self.can_partial_withdraw = can_partial_withdraw
        self.min_date_close_account = min_date_close_account
     
    def calc_total_summa(self):
        summa = self.summa
        year_percent = self.percent
        term_months = self.term
        return 'Общая сумма депозита: ' + str(
            summa * (1 + (year_percent/100/12))**term_months 
        )
    
    # 3. подсчет финальной суммы с учетом ежемесячной капитализации, 
    # которую получит вкладчик, если по истечение половины срока депозита, 
    # снимет n-ю долю со счета (параметр n должен передаться в метод как аргумент функции)
    
    def calc_total_summa_halfterm(self, n):
        if self.can_partial_withdraw != True:
            print ('Депозит {} не поддерживает частичное снятие'.format(self.name))
            return None
        
        else:
            summa = self.summa
            year_percent = self.percent
            term_months = self.term
        
            # сколько накопилось за половину срока
            t = term_months/2
            half_term_amount = summa * (1 + (year_percent/100/12))**t 
       
            # сколько снимается с вклада
            withdrawal_amount = half_term_amount * n 
        
            # сколько остается на вкладе после снятия
            remain_amount = half_term_amount - withdrawal_amount
        
            # продолжается накопление на оставшуюся сумму еще половину срока
            accumulation = remain_amount * (1 + (year_percent/100/12))**t 
        
            return accumulation    

In [103]:
dep5 = Deposit(12, 'anna', 'ivanova', 100000, 5, True, True, '2019-08-01')

In [104]:
print (dep5.calc_total_summa())

Общая сумма депозита: 105116.1897881733


In [105]:
dep5.calc_total_summa_halfterm(0.3)

73581.33285172132

In [106]:
dep6 = Deposit(12, 'anna', 'ivanova', 100000, 5, True, False, '2019-08-01')

In [107]:
dep6.calc_total_summa_halfterm(0.3)

Депозит anna не поддерживает частичное снятие
